In [1]:
from model import Encoder, Decoder, Seq2Seq
from data_loader import *
import pandas as pd
import torch.optim.lr_scheduler as lr_scheduler
from torch import optim
import torch.nn.functional as F
import datetime
import pretty_midi
import glob

In [2]:
import os
import matplotlib
import math
matplotlib.use('Agg')
# matplotlib.use("QtAgg")
import ffmpeg
#conda install -c conda-forge ffmpeg-python

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, writers
plt.rcParams['animation.ffmpeg_path'] = '/home/ilc/anaconda3/bin/ffmpeg'#'/usr/bin/ffmpeg'

import numpy as np
import subprocess as sp
from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy.audio.io.AudioFileClip import AudioFileClip

from midi2audio import FluidSynth

from torch.autograd import Variable 

In [3]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [4]:
dataset_name_path = f"./midi_list.txt"
dataloader = get_dataloader(dataset_name_path, batch_size=128) #[20, 512, 128], [20, 512, 102]

val_dataset_name_path = f"./midi_list_eval.txt"
val_dataloader = get_val_dataloader(val_dataset_name_path, batch_size=128) #[20, 512, 128], [20, 512, 102]

learning_rate = 0.001#0.001

input_size_encoder = 128 #129 #128
input_size_decoder = 102 #102 #24
output_size = 102#102 #24

# encoder_embedding_size = 300
# decoder_embedding_size = 300
enc_dropout = 0.5
dec_dropout = 0.
step = 0

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

self.piece_count:  105
dataset_len:  10500
val_dataset_len 5
cuda:0


In [5]:
class LSTM1(nn.Module):
    def __init__(self, output_dim, input_size, hidden_size, num_layers, seq_length):
        super(LSTM1, self).__init__()
        self.output_dim = output_dim #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True) #lstm
        self.fc_1 =  nn.Linear(hidden_size, output_dim) #fully connected to determine output dim

        self.relu = nn.ReLU()

    def forward(self,x):
        # h0, c0 no time information
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device) #hidden state
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device) #internal state
        # Propagate input through LSTM
        # x is MIDI => [44, 512, 128]

        # hn is final state, run over the sequence length
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
        # hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
        # print("output.shape", output.shape)
        # print("hn.shape", hn.shape)
        # out = self.relu(hn)
        out = self.fc_1(output) #final
        return out
 

In [6]:
# Define the model architecture
input_size = 128 #number of features
hidden_size = 1024 #number of features in hidden state
num_layers = 1 #number of stacked lstm layers
seq_len = 512
output_dim = 102 #number of output classes

# model = LSTM(vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate, tie_weights).to(device)
# model = LSTM(embedding_dim, hidden_dim, num_layers, dropout_rate, tie_weights).to(device)
model = LSTM1(output_dim, input_size, hidden_size, num_layers, seq_len).to(device) #our lstm class
model.train()

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2)

num_epochs = 100 #10
avg_loss_list = []
all_loss_list = []
val_loss_per_epoch_list = []

In [7]:
def customized_mse_loss(output, target, prev_output, midi_array):
    # target = target.transpose(0, 1)
    # print("output", output)
    # print(output.shape) #torch.Size([20, 513, 102])
    # print(target.shape) #torch.Size([20, 513, 102])
    mse_loss = F.mse_loss(output, target)
    # print("mse_loss:", mse_loss)

    var_diff = torch.var(torch.squeeze(output), dim=1, keepdim=True)
    mean_diff = torch.mean(var_diff)
    
    # Condition 1: Penalize if output is similar to previous output
    if mean_diff < 1e-4: #threshold
        #output [512, 1, 102] => [102] <-> [102] <-> [102] <-> ... <-> [102]
        mse_loss *= 1000

    # Condition 2: Stop movement if input is all zeros
    # midi_transpose = midi_array.transpose(0, 1)
    # midi_sum_row = torch.sum(midi_transpose, dim=-1)
    # mask = midi_sum_row == 0
    # mask = mask.unsqueeze(-1)
    # mask = mask.to(device)
    # # according to recorded index, make a mask [0, 1, 1, 0, ..., 0], true part will be omit(set value to 0).
    # # before compute mse, use mask first to tensor, then caculate MES loss
    # masked_output = output.masked_fill(mask, 0) #inplace function
    # masked_target = target.masked_fill(mask, 0)
    # mse_loss += F.mse_loss(masked_output, masked_target) * 100 #output 和 previous output 不像的話，增大 loss

    # Condition 3: Penalize if right-hand movement is too different between outputs
    # if output.shape[-1] == 21:  # Assumes hand joints are the last 21 dimensions
    #     rh_indices = [i for i in range(12, 21)]  # Right-hand joint indices
    #     rh_output = output[..., rh_indices]
    #     rh_prev_output = prev_output[..., rh_indices]
    #     rh_loss = nn.functional.mse_loss(rh_output, rh_prev_output)
    #     if rh_loss > 0.1:
    #         mse_loss *= 1000

    return mse_loss

In [8]:
def evaluate_lstm(model, val_dataloader):
    model.eval()
    print('Validation')
    valid_running_loss = 0.0
    counter = 0
    # previous_output = torch.zeros(512, 102).to(device)
    with torch.no_grad():
        for i, (inputs, targets) in enumerate(val_dataloader): #tqdm(enumerate(val_dataloader), total=len(val_dataloader))
            counter += 1

            inputs = inputs.to(device).float()
            targets = targets.to(device).float()
            print("inputs.shape:", inputs.shape)
            print("targets.shape:", targets.shape)
            outputs = model(inputs)
            print("outputs.shape:", outputs.shape)

            loss =  F.mse_loss(outputs, targets)
            valid_running_loss += loss.cpu().item()
            # previous_output = outputs

    epoch_val_loss = valid_running_loss / counter
    return epoch_val_loss

In [9]:
def evaluate_l1(model, val_dataloader):
    model.eval()
    print('Validation L1')
    valid_running_loss = 0.0
    counter = 0
    # previous_output = torch.zeros(512, 102).to(device)
    with torch.no_grad():
        for i, (inputs, targets) in enumerate(val_dataloader): #tqdm(enumerate(val_dataloader), total=len(val_dataloader))
            counter += 1

            inputs = inputs.to(device).float()
            targets = targets.to(device).float()
            print("inputs.shape:", inputs.shape)
            print("targets.shape:", targets.shape)
            outputs = model(inputs)
            print("outputs.shape:", outputs.shape)

            loss =  F.l1_loss(outputs, targets)
            valid_running_loss += loss.cpu().item()
            # previous_output = outputs

    epoch_val_loss = valid_running_loss / counter
    return epoch_val_loss

In [10]:
# train the model
for epoch in range(num_epochs):
    # previous_output = torch.zeros(1, 512, 102).to(device)
    losses = []
    for i, (midi_batch, motion_batch) in enumerate(dataloader):
        model.train()
        
        midi_batch = midi_batch.to(device).float()
        motion_batch = motion_batch.to(device).float()
        # print("midi_batch", midi_batch.shape)
        # print("motion_batch", motion_batch.shape)

        optimizer.zero_grad()
        output = model(midi_batch) #midi_batch
        # print("output.shape", output.shape)

        # motion_ground_truth_padding = F.pad(motion_batch, (0,0,0,1), value = 1) #<eot>
        
        # loss =  F.mse_loss(output, motion_ground_truth_padding)
        loss =  F.mse_loss(output, motion_batch)
        # loss = customized_mse_loss(output, motion_ground_truth_padding, previous_output, midi_batch)
        # loss = customized_mse_loss(output, motion_batch, previous_output, midi_batch)

        # losses 累計lose
        losses.append(loss.cpu().item())
        all_loss_list.append(loss.cpu().item())
        loss.backward()

        optimizer.step()
        mean_loss = sum(losses)/len(losses)

        print(f"Epoch {epoch}, batch {i}: loss = {loss.cpu().item():.6f}")

        # scheduler.step(1)
        # previous_output = output

        loc_dt = datetime.datetime.today()
        loc_dt_format = loc_dt.strftime("%Y-%m-%d_%H-%M-%S")

    val_loss = evaluate_lstm(model, val_dataloader) #CUDA out of memory
    val_loss_per_epoch_list.append(val_loss)
    print(f"Epoch {epoch}: val_loss = {val_loss:.6f}")
    # save_best_model(
    #         val_loss, epoch, model, optimizer, loss, loc_dt_format, mean_loss
    #     )
    avg_loss_list.append(mean_loss)
    loc_dt = datetime.datetime.today()
    loc_dt_format = loc_dt.strftime("%Y-%m-%d_%H-%M-%S")
    if (epoch+1)%100 == 0:
        torch.save({
            'epoch':epoch,
            'model_state_dict':model.state_dict(),
            'optimizer_state_dict':optimizer.state_dict(),
            'loss':loss
        }, "./model_save/[100epoch]LSTM_save_epoch_" + str(epoch)+ "_"+ str(loc_dt_format) + "_avg_loss_" + str(mean_loss) +".tar")

Epoch 0, batch 0: loss = 0.179273
Epoch 0, batch 1: loss = 0.172937
Epoch 0, batch 2: loss = 0.166642
Epoch 0, batch 3: loss = 0.159391
Epoch 0, batch 4: loss = 0.148991
Epoch 0, batch 5: loss = 0.133344
Epoch 0, batch 6: loss = 0.113596
Epoch 0, batch 7: loss = 0.079768
Epoch 0, batch 8: loss = 0.103472
Epoch 0, batch 9: loss = 0.040354
Epoch 0, batch 10: loss = 0.042724
Epoch 0, batch 11: loss = 0.041152
Epoch 0, batch 12: loss = 0.035520
Epoch 0, batch 13: loss = 0.026343
Epoch 0, batch 14: loss = 0.015222
Epoch 0, batch 15: loss = 0.008446
Epoch 0, batch 16: loss = 0.009299
Epoch 0, batch 17: loss = 0.014510
Epoch 0, batch 18: loss = 0.014134
Epoch 0, batch 19: loss = 0.010597
Epoch 0, batch 20: loss = 0.007210
Epoch 0, batch 21: loss = 0.005844
Epoch 0, batch 22: loss = 0.006404
Epoch 0, batch 23: loss = 0.007502
Epoch 0, batch 24: loss = 0.008244
Epoch 0, batch 25: loss = 0.008608
Epoch 0, batch 26: loss = 0.008079
Epoch 0, batch 27: loss = 0.007045
Epoch 0, batch 28: loss = 0.00

In [11]:
print(loc_dt_format)
print(avg_loss_list)

2023-07-18_13-54-24
[0.0213198921143874, 0.0026611532337110803, 0.0024372325727648764, 0.0022988429584776065, 0.0022155684643391385, 0.00214571491759046, 0.0020854319866567134, 0.0020437723401574843, 0.0019774749434654073, 0.0019260541022564452, 0.0018811354607186582, 0.0018347022951165416, 0.001801472263837076, 0.0017889463539089423, 0.0017671494138797364, 0.0017520131400214083, 0.0017478895003357566, 0.0017237138193863702, 0.0017384010751406293, 0.0017329152373327728, 0.0017162516898284834, 0.0017188132771414268, 0.001697643328996667, 0.0016883854959047883, 0.0017093650769464761, 0.001694144752628383, 0.0016916720392967921, 0.0016664323994485908, 0.0016627982328926403, 0.0016543447090514812, 0.0016582072684429138, 0.0016527718917679895, 0.0016399727186120778, 0.0016522905999704837, 0.0016566956225019622, 0.001636327025536672, 0.0016422759592869733, 0.0016434365218639913, 0.0016451326496225882, 0.001604169504773096, 0.001604637104445642, 0.0016197713178930332, 0.0016144267928573381, 0

In [12]:
print(val_loss_per_epoch_list)

[0.024104325100779533, 0.02754058688879013, 0.027194347232580185, 0.026935724541544914, 0.026952262967824936, 0.02684026025235653, 0.026775600388646126, 0.026811813935637474, 0.02688557654619217, 0.02740233950316906, 0.02663080208003521, 0.02690023183822632, 0.02715427428483963, 0.026969920843839645, 0.027722341939806938, 0.027760889381170273, 0.027627751231193542, 0.027202337980270386, 0.026781050488352776, 0.02730695903301239, 0.02671784907579422, 0.02705729566514492, 0.027684679254889488, 0.028540076687932014, 0.028338579460978508, 0.027843404561281204, 0.027968639507889748, 0.02795819193124771, 0.027818838134407997, 0.02792397513985634, 0.027266284450888634, 0.02696794830262661, 0.027149623259902, 0.027810048311948776, 0.02772761508822441, 0.027289971709251404, 0.028640104457736015, 0.028583869338035583, 0.027218086645007133, 0.027341673150658607, 0.027668027207255363, 0.02772059477865696, 0.026857182383537292, 0.026569096371531487, 0.0269019678235054, 0.02654779516160488, 0.027736

In [13]:
# def lr_lambda(epoch):
#     # LR to be 0.1 * (1/1+0.01*epoch)
#     base_lr = 0.1
#     factor = 0.01
#     return base_lr/(1+factor*epoch)

In [14]:
# scheduler = lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.3, total_iters=10)
# scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda)

In [15]:
plt.cla()
plt.clf()

In [16]:
print(len(avg_loss_list))
avg_loss_list_dataframe = pd.DataFrame(avg_loss_list)

100


In [17]:
avg_loss_list_dataframe

,0
0,0.021320
1,0.002661
2,0.002437
3,0.002299
4,0.002216
...,...
95,0.001259
96,0.001261
97,0.001240
98,0.001237


In [18]:
plt.plot(np.array(avg_loss_list_dataframe.index), np.array(avg_loss_list_dataframe[0]))
plt.savefig("avg_loss_training.jpg")
plt.show()

In [19]:
plt.cla()
plt.clf()

In [20]:
loss_list_dataframe = pd.DataFrame(all_loss_list)

In [21]:
plt.plot(np.array(loss_list_dataframe.index), np.array(loss_list_dataframe[0]))
plt.savefig("training_loss.jpg")
plt.show()

In [22]:
plt.cla()
plt.clf()

In [23]:
val_loss_per_epoch_list_dataframe = pd.DataFrame(val_loss_per_epoch_list)

In [24]:
plt.plot(np.array(val_loss_per_epoch_list_dataframe.index), np.array(val_loss_per_epoch_list_dataframe[0]))
plt.savefig("training_val_loss.jpg")
plt.show()

In [25]:
def predict(model, input, target, device):
    model.eval()
    with torch.no_grad():
        input = torch.as_tensor(input).to(torch.float32).to(device)
        print(target.shape)
        target = torch.as_tensor(target).to(torch.float32).to(device)
        # TODO: target should be <sos>, should not random
        outputs = model(input)
        return outputs.cpu().numpy()

In [26]:
def read_midi(filename, specific_fps):
    # Load the MIDI file
    midi_data = pretty_midi.PrettyMIDI(filename)

    piano_roll = midi_data.get_piano_roll(fs=specific_fps)  # 40fps #250fps
    piano_roll[piano_roll > 0] = 1

    return piano_roll

In [27]:
test_datapath = "./BWV1001/"
change_fps = 40
test_midi_path_list = glob.glob(test_datapath + "*.mid")
test_data_list = []
test_music_list = []
for test_midi in test_midi_path_list:
    str_name = test_midi
    print("str_name:", str_name)
    filename = str_name.split('/')[2]
    filecode = filename.split('.')[0]
    print("filecode: ",filecode)
    test_music_list.append(filecode)
    
    print(test_midi)
    read_piano_roll = read_midi(test_midi, change_fps)
    read_piano_roll_transpose = read_piano_roll.T
    print(read_piano_roll_transpose.shape)
    test_midi_len = read_piano_roll_transpose.shape[0]
    test_data_list.append(read_piano_roll_transpose)

str_name: ./BWV1001/vs1-1ada.mid
filecode:  vs1-1ada
./BWV1001/vs1-1ada.mid
(8171, 128)
str_name: ./BWV1001/vs1-2fug.mid
filecode:  vs1-2fug
./BWV1001/vs1-2fug.mid
(11537, 128)
str_name: ./BWV1001/vs1-3sic.mid
filecode:  vs1-3sic
./BWV1001/vs1-3sic.mid
(6993, 128)
str_name: ./BWV1001/vs1-4prs.mid
filecode:  vs1-4prs
./BWV1001/vs1-4prs.mid
(7897, 128)


In [28]:
def column(matrix, i):
    return [row[i] for row in matrix]

def test_render_animation(fps, output, azim, prediction, ground_truth=None):
    prediction_array = np.asarray(prediction)
    print(prediction_array.size)
    limit = len(prediction_array)
    print("limit", limit)
    size = 6#6
    fps = 40

    # Skeleton layout
    parents = [[0, 1], [1, 3], [3, 2], [0, 2],#head
                [8, 6], [6, 13], [13, 4], [4, 8],#shoulder
                [6, 4], [4, 5], [5, 7], [7, 6],#Upper torso
                [8, 18], [8, 20], [13, 21], [13, 19],
                [5, 20], [5, 21], [7, 18], [7, 19],
                [18, 19], [19, 21], [21, 20], [20, 18], #waist
                [18, 22], [20, 22], [22, 23], [22, 25], [23, 25], [24,23], [24, 25],  #right lag
                [21, 26], [19, 26], [26, 27], [26, 29], [27, 29], [28, 27], [28, 29], #left lag
                [8, 9], [9, 11], [9, 10], [10, 11], [10, 12], [9, 12], [11, 12], #right hand
                [13, 14], [14, 16], [14, 15], [16, 15], [14, 17], [16, 17], [15, 17], #left hand
                [31, 33], [30, 32], [30, 31], [32, 33], [31, 32], [30, 33] #instrument
                        ]
    # joints_right = [1, 2, 12, 13, 14]

    prediction_array[:, :, 2] += 0.1 #[:, :, 2]
    if ground_truth is not None:
        ground_truth[:, :, 2] += 0.1
        poses = {'Prediction': prediction_array,
                 'Ground_truth': ground_truth}
    else:
        poses = {'Prediction': prediction_array}
    

    fig = plt.figure()#(figsize=(size*len(poses), size))
    # ax_3d = []
    # lines_3d = []
    radius = 1#14 #3.7#
    # print(poses)
    for index, (title, data) in enumerate(poses.items()):
        ax = fig.add_subplot(1, len(poses), index + 1, projection='3d')
        ax.clear()
        print(data)
        ims = [] #每一 frame 都存
        for frame_index, each_frame in enumerate(data):
            # print("each_frame")
            # print(each_frame)
            ax.view_init(elev=15., azim=azim)
            ax.set_xlim3d([-radius/2, radius/2])
            ax.set_zlim3d([0, radius])
            ax.set_ylim3d([-radius/2, radius/2])
            ax.set_aspect('auto') #ax.set_aspect('equal')

            # print(title)
            points = ax.scatter(column(each_frame[:30], 0), column(each_frame[:30], 1), column(each_frame[:30], 2), cmap='jet', marker='o', label='body joint', color = 'black')
            points_2 = ax.scatter(column(each_frame[30:32], 0), column(each_frame[30:32], 1), column(each_frame[30:32], 2), cmap='jet', marker='o', label='body joint', color = 'blue')
            points_3 = ax.scatter(column(each_frame[32:34], 0), column(each_frame[32:34], 1), column(each_frame[32:34], 2), cmap='jet', marker='o', label='body joint', color = 'red')
            
            # ax.scatter(column(each_frame, 0), column(each_frame, 1), column(each_frame, 2), cmap='jet', marker='o', label='body joint')
            # ax.legend()
            # print("+++")
            
            parents = [[0, 1], [1, 3], [3, 2], [0, 2],#head
                        [8, 6], [6, 13], [13, 4], [4, 8],#shoulder
                        [6, 4], [4, 5], [5, 7], [7, 6],#Upper torso
                        [8, 18], [8, 20], [13, 21], [13, 19],
                        [5, 20], [5, 21], [7, 18], [7, 19],
                        [18, 19], [19, 21], [21, 20], [20, 18], #waist
                        [18, 22], [20, 22], [22, 23], [22, 25], [23, 25], [24,23], [24, 25],  #right lag
                        [21, 26], [19, 26], [26, 27], [26, 29], [27, 29], [28, 27], [28, 29], #left lag
                        [8, 9], [9, 11], [9, 10], [10, 11], [10, 12], [9, 12], [11, 12], #right hand
                        [13, 14], [14, 16], [14, 15], [16, 15], [14, 17], [16, 17], [15, 17], #left hand
                        [30, 31], [32, 33],  #instrument
                        # [31, 33], [30, 32], [30, 31], [32, 33], [31, 32], [30, 33] #instrument
                        ]
            lines = []
            # draw line
            
            # lines = [ax.plot([each_frame[vs][0], each_frame[ve][0]],
            #                  [each_frame[vs][1], each_frame[ve][1]],
            #                  [each_frame[vs][2], each_frame[ve][2]]) for (vs, ve) in parents]
            line_num = len(parents)
            for idx, each_line in enumerate(parents):
                vec_start = each_frame[each_line[0]]
                vec_end = each_frame[each_line[1]]
                # print(vec_start)
                # print(vec_end)
                line_color = "black"
                if idx == line_num-2:
                    line_color = "blue"
                if idx == line_num-1:
                    line_color = "red"
                # ax.plot([vec_start[0], vec_end[0]], [vec_start[1], vec_end[1]], [vec_start[2], vec_end[2]])
                
                temp, = ax.plot([vec_start[0], vec_end[0]], [vec_start[1], vec_end[1]], [vec_start[2], vec_end[2]], color=line_color)
                lines.append(temp)

            # ax.figure.savefig('./test_pic/pic' + str(frame_index) + '.png', dpi=100, bbox_inches = 'tight')

            # ims.append([points])
            # image_frame = [points].extend(lines)
            ims.append([points]+[points_2]+[points_3]+lines) #TODO: try extend

            # plt.cla()
            # print("+++")

    anim = matplotlib.animation.ArtistAnimation(fig, ims, interval=1000/fps)

    if output.endswith('.mp4'):
        FFwriter = matplotlib.animation.FFMpegWriter(fps=fps, extra_args=['-vcodec', 'libx264'])
        anim.save(output, writer=FFwriter)
    elif output.endswith('.gif'):
        anim.save(output, fps=fps, dpi=100, writer='imagemagick')
    else:
        raise ValueError('Unsupported output format (only .mp4 and .gif are supported)')

In [29]:
def plot(audio_path, plot_path, prediction, sample_time, fps, name=""): #audio_path, plot_path, 
    # render_animation(fps, output='new_temp.mp4', azim=75, prediction=prediction)
    test_render_animation(fps, output='new_temp_' + name + '.mp4', azim=75, prediction=prediction)

    # # #merge with wav
    input_video = ffmpeg.input('new_temp_' + name + '.mp4')
    fluid_syn = FluidSynth()
    fluid_syn.midi_to_audio(audio_path, './output' + name + '.wav')
    input_audio = ffmpeg.input('./output' + name + '.wav')
    # output = ffmpeg.output(video, audio, plot_path, vcodec='copy', acodec='aac', strict='experimental')
    ffmpeg.concat(input_video, input_audio, v=1, a=1).output(plot_path).run()
    # os.remove('new_temp_' + name + '.mp4')

In [30]:
model.eval()

full_prediction = pd.DataFrame()
num_count = 0
# read midi
# test_dataloader = get_dataloader(test_datapath, batch_size=1)
for test_batch in test_data_list:
    with torch.no_grad():
        first_target = torch.zeros(test_batch.shape[0],102)
        print(first_target.shape)
        test_input = test_batch[None, :]
        test_target = first_target[None, :]
        print("test_input", test_input.shape)
        print("test_target", test_target.shape)
        prediction = predict(model, test_input, test_target, device)
        
        # print(prediction.shape)
        
        prediction  = prediction[:, :, :102]
        print("prediction.shape", prediction.shape)
        
        # full_prediction.append(prediction)
        full_prediction = pd.DataFrame(prediction[0])
        print("full_prediction", full_prediction.shape)
        
        # prev_prediction = prediction[0][:-1][None, :]
        # print(prev_prediction.shape)
        
        Row_list_prediction =[]
        
        filecode = test_music_list[num_count]
    
        # Iterate over each row
        for index, rows in full_prediction.iterrows():
            #fill nan
            rows = rows.fillna(0)
            # Create list for the current row
            my_list = rows.values.tolist()
            # print(my_list)
            
            my_list_per3 = [my_list[i:i+3] for i in range(0, len(my_list), 3)]
            # append the list to the final list
            Row_list_prediction.append(my_list_per3)

        # print(len(Row_list_prediction), len(Row_list_prediction[0]),len(Row_list_prediction[0][0]))
        plot(test_datapath + test_music_list[num_count] + ".mid", "./video_" + filecode + "_test_predict.mp4", Row_list_prediction[:800], None, 40, filecode) #ow_list[0:900]
        # print("prediction.shape", prediction.shape)
        prediction_arr = np.array(Row_list_prediction)
        # formated_motion = prediction_format(full_prediction)
        # # # plot(formated_motion)
        # audio_path = test_music_list[num_count][0]
        # output_path = "test_output_" + filecode + ".mp4"
        # plot(formated_motion, audio_path, output_path, None, 10, filecode)
        num_count += 1

# model.train()

torch.Size([8171, 102])
test_input (1, 8171, 128)
test_target torch.Size([1, 8171, 102])
torch.Size([1, 8171, 102])
prediction.shape (1, 8171, 102)
full_prediction (8171, 102)
81600
limit 800
[[[ 7.63824657e-02  1.43785849e-01  1.10197017e+00]
  [ 1.66326743e-02  8.82398188e-02  1.11780629e+00]
  [ 1.18391097e-01  6.94648772e-02  1.08783660e+00]
  ...
  [ 2.05493141e-02  1.05169207e-01  9.81932378e-01]
  [-1.32544398e-01  6.86568990e-02  1.09138844e+00]
  [ 8.44395086e-02  2.68868953e-01  7.53729200e-01]]

 [[ 8.12238306e-02  1.41789436e-01  1.09309480e+00]
  [ 2.05650218e-02  9.24802423e-02  1.11217258e+00]
  [ 1.30981937e-01  7.08705634e-02  1.08539257e+00]
  ...
  [ 2.52187476e-02  1.02595560e-01  9.78103793e-01]
  [-1.34262621e-01  7.60225207e-02  1.09813068e+00]
  [ 8.92863348e-02  2.65725225e-01  7.47544146e-01]]

 [[ 8.46444219e-02  1.40322179e-01  1.08762619e+00]
  [ 2.33595595e-02  9.45660993e-02  1.10743127e+00]
  [ 1.35127887e-01  7.26895779e-02  1.08215568e+00]
  ...
  [ 2.

fluidsynth: panic: An error occurred while reading from stdin.


FluidSynth runtime version 2.1.1
Copyright (C) 2000-2020 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of E-mu Systems, Inc.

Rendering audio to file './outputvs1-1ada.wav'..


ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/ilc/anaconda3/envs/sinica --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  5.100 /  5.  5.100
  libswresample   3.  5.100 /  3.  5.100
  libpostproc    55

torch.Size([11537, 102])
test_input (1, 11537, 128)
test_target torch.Size([1, 11537, 102])
torch.Size([1, 11537, 102])
prediction.shape (1, 11537, 102)
full_prediction (11537, 102)
81600
limit 800
[[[ 0.06097613  0.13510084  1.07036445]
  [-0.00165589  0.08713032  1.0835422 ]
  [ 0.10868047  0.06488154  1.06048439]
  ...
  [ 0.00639743  0.1064601   0.95100693]
  [-0.13132824  0.09068167  1.06366823]
  [ 0.07663165  0.25426248  0.72863308]]

 [[ 0.06387136  0.13782644  1.09621648]
  [ 0.00145942  0.08780219  1.11036668]
  [ 0.11160298  0.06789427  1.08956847]
  ...
  [ 0.00877939  0.10496889  0.97695867]
  [-0.1346896   0.0994775   1.0960888 ]
  [ 0.07719963  0.2575891   0.74549911]]

 [[ 0.06433575  0.13730524  1.09602675]
  [ 0.00167201  0.08934662  1.11087511]
  [ 0.11247785  0.06772277  1.09027383]
  ...
  [ 0.00928321  0.10501685  0.97669492]
  [-0.13371043  0.10298171  1.09909127]
  [ 0.0764659   0.25569034  0.74560473]]

 ...

 [[ 0.04377478  0.14961687  1.09522144]
  [-0.016058

...

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/ilc/anaconda3/envs/sinica --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  5.100 /  5.  5.100
  libswresample   3.  5.100 /  3.  5.100
  libpostproc    55

torch.Size([7897, 102])
test_input (1, 7897, 128)
test_target torch.Size([1, 7897, 102])
torch.Size([1, 7897, 102])
prediction.shape (1, 7897, 102)
full_prediction (7897, 102)
81600
limit 800
[[[ 6.70695379e-02  1.32236868e-01  1.08773515e+00]
  [ 1.64203160e-03  8.08354765e-02  1.10022411e+00]
  [ 1.11160442e-01  6.08248524e-02  1.07508186e+00]
  ...
  [ 1.11286137e-02  1.06361628e-01  9.67465913e-01]
  [-1.23309761e-01  9.76053849e-02  1.08723626e+00]
  [ 7.73368329e-02  2.47672349e-01  7.25071287e-01]]

 [[ 6.44638613e-02  1.31471589e-01  1.09542439e+00]
  [ 2.12363247e-03  7.89708644e-02  1.10682068e+00]
  [ 1.12228617e-01  6.10676296e-02  1.08570395e+00]
  ...
  [ 1.04235727e-02  1.02368616e-01  9.75429273e-01]
  [-1.25490457e-01  1.06093965e-01  1.10379980e+00]
  [ 7.30526149e-02  2.41377980e-01  7.25882626e-01]]

 [[ 6.25389293e-02  1.30805314e-01  1.09476123e+00]
  [ 5.51138073e-04  8.02002251e-02  1.10646162e+00]
  [ 1.11474797e-01  6.13100529e-02  1.08694181e+00]
  ...
  [ 9.

fluidsynth: panic: An error occurred while reading from stdin.


FluidSynth runtime version 2.1.1
Copyright (C) 2000-2020 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of E-mu Systems, Inc.

Rendering audio to file './outputvs1-4prs.wav'..


ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/ilc/anaconda3/envs/sinica --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  5.100 /  5.  5.100
  libswresample   3.  5.100 /  3.  5.100
  libpostproc    55

In [31]:
model.eval()

full_prediction = pd.DataFrame()
num_count = 0
# read midi
# test_dataloader = get_dataloader(test_datapath, batch_size=1)
for test_batch in test_data_list:
    with torch.no_grad():
        first_target = torch.zeros(test_batch.shape[0],102)
        print(first_target.shape)
        test_input = test_batch[None, :]
        test_target = first_target[None, :]
        print("test_input", test_input.shape)
        print("test_target", test_target.shape)
        prediction = predict(model, test_input, test_target, device)
        
        # print(prediction.shape)
        
        prediction  = prediction[:, :, :102]
        print("prediction.shape", prediction.shape)
        
        # full_prediction.append(prediction)
        full_prediction = pd.DataFrame(prediction[0])
        print("full_prediction", full_prediction.shape)
        
        # prev_prediction = prediction[0][:-1][None, :]
        # print(prev_prediction.shape)
        
        Row_list_prediction =[]
        
        filecode = test_music_list[num_count]
    
        # Iterate over each row
        for index, rows in full_prediction.iterrows():
            #fill nan
            rows = rows.fillna(0)
            # Create list for the current row
            my_list = rows.values.tolist()
            # print(my_list)
            
            my_list_per3 = [my_list[i:i+3] for i in range(0, len(my_list), 3)]
            # append the list to the final list
            Row_list_prediction.append(my_list_per3)

        prediction_arr = np.array(Row_list_prediction)
        if not os.path.exists('./output_prediction/[midi_no_annotation]'+str(num_layers)+'LSTM_hidden'+str(hidden_size)+'_'+str(num_epochs)+'epoch/'):
            os.makedirs('./output_prediction/[midi_no_annotation]'+str(num_layers)+'LSTM_hidden'+str(hidden_size)+'_'+str(num_epochs)+'epoch/')
        midi_data_output = open('./output_prediction/[midi_no_annotation]'+str(num_layers)+'LSTM_hidden'+str(hidden_size)+'_'+str(num_epochs)+'epoch/prediction_'+
                                filecode +'.pkl', 'wb')
        pickle.dump(prediction_arr, midi_data_output)
        midi_data_output.close()
        
        num_count += 1

# model.train()

torch.Size([8171, 102])
test_input (1, 8171, 128)
test_target torch.Size([1, 8171, 102])
torch.Size([1, 8171, 102])
prediction.shape (1, 8171, 102)
full_prediction (8171, 102)
torch.Size([11537, 102])
test_input (1, 11537, 128)
test_target torch.Size([1, 11537, 102])
torch.Size([1, 11537, 102])
prediction.shape (1, 11537, 102)
full_prediction (11537, 102)
torch.Size([6993, 102])
test_input (1, 6993, 128)
test_target torch.Size([1, 6993, 102])
torch.Size([1, 6993, 102])
prediction.shape (1, 6993, 102)
full_prediction (6993, 102)
torch.Size([7897, 102])
test_input (1, 7897, 128)
test_target torch.Size([1, 7897, 102])
torch.Size([1, 7897, 102])
prediction.shape (1, 7897, 102)
full_prediction (7897, 102)


In [32]:
final_val_loss = evaluate_lstm(model, val_dataloader)

Validation
len(midi_data) 6069
len(motion_data) 6069
len(midi_data) 6706
len(motion_data) 6706
len(midi_data) 4525
len(motion_data) 4525
len(midi_data) 6061
len(motion_data) 6061
len(midi_data) 5281
len(motion_data) 5281
inputs.shape: torch.Size([5, 6706, 128])
targets.shape: torch.Size([5, 6706, 102])
outputs.shape: torch.Size([5, 6706, 102])


In [33]:
print(final_val_loss)

0.026906190440058708


In [34]:
final_l1_loss = evaluate_l1(model, val_dataloader)
print(final_l1_loss)

Validation L1
len(midi_data) 6069
len(motion_data) 6069
len(midi_data) 4525
len(motion_data) 4525
len(midi_data) 5281
len(motion_data) 5281
len(midi_data) 6706
len(motion_data) 6706
len(midi_data) 6061
len(motion_data) 6061
inputs.shape: torch.Size([5, 6706, 128])
targets.shape: torch.Size([5, 6706, 102])
outputs.shape: torch.Size([5, 6706, 102])
0.06567662209272385
